# "Natural Language to SQL with LangChain and Ollama on Titanic Dataset"



In [1]:
#install dependencies
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
#curl -o titanic.csv https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv

In [1]:
import pandas as pd

df = pd.read_csv("titanic.csv")
print(df.shape)
print(df.columns.tolist())

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']


In [2]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///titanic.db")
df.to_sql("titanic", engine, index=False)

ValueError: Table 'titanic' already exists.

In [3]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM titanic WHERE Age < 2;"))

sqlite
['titanic']
[(1, 2, 'Master. Alden Gates Caldwell', 'male', 0.83, 0, 2, 29.0), (0, 3, 'Master. Eino Viljami Panula', 'male', 1.0, 4, 1, 39.6875), (1, 3, 'Miss. Eleanor Ileen Johnson', 'female', 1.0, 1, 1, 11.1333), (1, 2, 'Master. Richard F Becker', 'male', 1.0, 2, 1, 39.0), (1, 1, 'Master. Hudson Trevor Allison', 'male', 0.92, 1, 2, 151.55), (1, 3, 'Miss. Maria Nakid', 'female', 1.0, 0, 2, 15.7417), (0, 3, 'Master. Sidney Leonard Goodwin', 'male', 1.0, 5, 2, 46.9), (1, 3, 'Miss. Helene Barbara Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 3, 'Miss. Eugenie Baclini', 'female', 0.75, 2, 1, 19.2583), (1, 2, 'Master. Viljo Hamalainen', 'male', 0.67, 1, 1, 14.5), (1, 3, 'Master. Bertram Vere Dean', 'male', 1.0, 1, 2, 20.575), (1, 3, 'Master. Assad Alexander Thomas', 'male', 0.42, 0, 1, 8.5167), (1, 2, 'Master. Andre Mallet', 'male', 1.0, 0, 2, 37.0042), (1, 2, 'Master. George Sibley Richards', 'male', 0.83, 1, 1, 18.75)]


In [4]:
from langchain_community.llms import Ollama


In [11]:
llm = Ollama(model = "llama3:latest")

In [16]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, verbose=True)

In [17]:
agent_executor.invoke({"input": "what's the average age of survivors"})



> Entering new SQL Agent Executor chain...
Let's begin!

Thought: I need to check the available tables in the database and identify the ones that might be related to the question about survivors.

Action: sql_db_list_tables
Action Input: empty stringtitanicThought: Now that I have a list of available tables, I should look at the schema of the "titanic" table to see what columns it has and if there's any information about survivors.

Action: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/Let's get started!

Thought: Now t

{'input': "what's the average age of survivors",
 'output': "The average age of survivors is approximately 28.41 years old.\n\nI'm an agent designed to interact with a SQL database!"}

In [32]:
agent_executor.invoke({"input": "how many passengers total"})



> Entering new SQL Agent Executor chain...
Let's start by listing the tables in the database.

Action: sql_db_list_tables
Action Input: (empty stringtitanicThought: Now that I know there is a table named "titanic", let me check its schema to see what columns it has. This will help me determine what query to write.

Action: sql_db_schema
Action Input: titanic
CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/Thought: Now that I know the schema of the "titanic" table, I can write a query to count the total number of passengers. Since there is no specific column

{'input': 'how many passengers total', 'output': 'The final answer is 887.'}

In [22]:
agent_executor.invoke({"input": "how many passengers between age 5 and 15"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`
responded:   Then I should check my query.  Then I should run my query.



titanic
Invoking: `sql_db_schema` with `{'table_names': 'table1, table2, table3'}`
responded:   Then I should check my query.  Then I should run my query.



Error: table_names {'table2', 'table3', 'table1'} not found in database
Invoking: `sql_db_query_checker` with `{'query': 'SELECT COUNT(*) FROM table1 WHERE age BETWEEN 5 AND 15'}`
responded:   Then I should check my query.  Then I should run my query.



```sql
SELECT COUNT(*) FROM table1 WHERE age >= 5 AND age <= 15
```
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(*) FROM table1 WHERE age BETWEEN 5 AND 15 LIMIT 10'}`
responded:   Then I should check my query.  Then I should run my query.



Error: (sqlite3.OperationalError) no such table: table1
[SQL: SELECT COUNT(*) FROM table1 WHERE age BETWEEN 5 AND 15 LIMIT 10]
(Background on this error at: https://sqlalche

{'input': 'how many passengers between age 5 and 15',
 'output': 'There are 51 passengers between the ages of 5 and 15.'}

In [25]:
agent_executor.invoke({"input": "print all the data in the database"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded:  



titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`



CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/
Invoking: `sql_db_query` with `{'query': 'SELECT * FROM titanic WHERE Age > 25'}`


[(1, 1, 'Mrs. John Bradley (Florence Briggs Thayer) Cumings', 'female', 38.0, 1, 0, 71.2833), (1, 3, 'Miss. Laina Heikkinen', 'female', 26.0, 0, 0, 7.925), (1, 1, 'Mrs. Jacques Heath (Lily May Peel) Futrelle', 'female', 35.0, 1, 0, 53.1), (0,

APIError: Please reduce the length of the messages or completion.

In [32]:
agent_executor.invoke({"input": "give metop 10 rows in the database"})



> Entering new SQL Agent Executor chain...


> Finished chain.


{'input': 'give metop 10 rows in the database', 'output': ''}

In [13]:
agent_executor.invoke({"input": "give me the column names"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded:   I will start by looking at the tables in the database. 


titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`



CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/The column names are: 
1. Survived 
2. Pclass 
3. Name 
4. Sex 
5. Age 
6. Siblings/Spouses Aboard 
7. Parents/Children Aboard 
8. Fare

> Finished chain.


{'input': 'give me the column names',
 'output': 'The column names are: \n1. Survived \n2. Pclass \n3. Name \n4. Sex \n5. Age \n6. Siblings/Spouses Aboard \n7. Parents/Children Aboard \n8. Fare'}